In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [7]:
#Load all the raw data
amazon = pd.read_csv('datasets/amazon_prime_titles.csv')
disney = pd.read_csv('datasets/disney_plus_titles.csv')
hulu = pd.read_csv('datasets/hulu_titles.csv')
netflix = pd.read_csv('datasets/netflix_titles.csv')

In [8]:
datasets_names = os.listdir('datasets') #Get the filenames from the datasets to create the id column

In [9]:
for i, df in enumerate([amazon, disney, hulu, netflix]): #Loop through all the dataframes
    df['id'] = df.apply(lambda row: datasets_names[i][0] + row.show_id, axis=1) #Get the first letter of the df name and combine it with show id, apply this to every row.

In [10]:
#Concatenate all csvs into 1
df = pd.concat([amazon, disney, hulu, netflix], ignore_index=True)

In [11]:
df['rating'].fillna('G', inplace=True) #Replace all NA in rating column with a 'G' (general for all audiences)

In [12]:
#Change the date_added column to YYYY/mm/dd format
df['date_added'] = pd.to_datetime(df['date_added'])
df['date_added'] = df['date_added'].dt.strftime('%Y/%m/%d')

df['date_added'].head()

0    2021/03/30
1    2021/03/30
2    2021/03/30
3    2021/03/30
4    2021/03/30
Name: date_added, dtype: object

In [13]:
#For every object dtype column (string), use the lower() function to transform it to lowercase
for i in df.columns:
    if df[i].dtype == 'object':
        df[i] = df[i].str.lower()

df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021/03/30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,as1
1,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021/03/30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,as2
2,s3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021/03/30,2017,g,74 min,"action, drama, suspense",after a man discovers his wife is cheating on ...,as3
3,s4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021/03/30,2014,g,69 min,documentary,"pink breaks the mold once again, bringing her ...",as4
4,s5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021/03/30,1989,g,45 min,"drama, fantasy",teenage matt banting wants to work with a famo...,as5


In [14]:
#Split the duration column into 2 columns, one for the number and one for the type (minutes or seasons)
df[['duration_int', 'duration_type']] = df['duration'].str.split(' ', n=1, expand=True)
df['duration_type'].unique()

array(['min', 'season', 'seasons', nan], dtype=object)

In [15]:
#Change the duration_type where there is 'season' to 'seasons'
df['duration_type'] = df['duration_type'].replace('season', 'seasons')
df['duration_type'].unique()

array(['min', 'seasons', nan], dtype=object)

In [16]:
#Now we drop the replaced columns
df.drop(['duration', 'show_id'], axis=1, inplace=True)

In [17]:
df.head()

,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,id,duration_int,duration_type
0,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021/03/30,2014,g,"comedy, drama",a small fishing village must procure a local d...,as1,113,min
1,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021/03/30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,as2,110,min
2,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021/03/30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,as3,74,min
3,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021/03/30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",as4,69,min
4,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021/03/30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,as5,45,min


### Falta columna 'score' traida de ratings. Usar .map de pandas para poner el score promedio de cada pelicula del dataframe.

In [18]:
ratings_data1 = pd.read_csv("ratings/1.csv")
ratings_data2 = pd.read_csv("ratings/2.csv")
ratings_data3 = pd.read_csv("ratings/3.csv")
ratings_data4 = pd.read_csv("ratings/4.csv")
ratings_data5 = pd.read_csv("ratings/5.csv")
ratings_data6 = pd.read_csv("ratings/6.csv")
ratings_data7 = pd.read_csv("ratings/7.csv")
ratings_data8 = pd.read_csv("ratings/8.csv")

In [19]:
ratings = pd.concat([ratings_data1, ratings_data2, ratings_data3, ratings_data4, ratings_data5, ratings_data6, ratings_data7, ratings_data8], ignore_index=True)
ratings

,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186
2,1,5.0,1425941523,hs2381
3,1,5.0,1425941546,ns3663
4,1,5.0,1425941556,as9500
...,...,...,...,...
11024284,124380,4.5,1196786159,ns5272
11024285,124380,2.5,1196786030,ns5492
11024286,124380,3.5,1196785679,hs305
11024287,124380,4.5,1196787089,ns7881


In [20]:
mean_ratings = round(ratings.groupby('movieId')['rating'].mean(), 2)
df['score'] = df['id'].map(mean_ratings)
df.head()

,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,id,duration_int,duration_type,score
0,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021/03/30,2014,g,"comedy, drama",a small fishing village must procure a local d...,as1,113,min,3.47
1,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021/03/30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,as2,110,min,3.55
2,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021/03/30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,as3,74,min,3.50
3,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021/03/30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",as4,69,min,3.54
4,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021/03/30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,as5,45,min,3.48


In [21]:
#We save the titles dataframe to a csv
df.to_csv('data_transformed.csv', index=False)

In [22]:
#We save the ratings dataframe to a csv
ratings.to_csv('ratings.csv', index=False)